In [1]:
!pip install pickle5

     |████████████████████████████████| 133kB 21.9MB/s 
  Created wheel for pickle5: filename=pickle5-0.0.11-cp37-cp37m-linux_x86_64.whl size=219249 sha256=dba74453cea1a1e8fc90025d86fa1480b5d8adb612bb354a56a2a930fbf86423
  Stored in directory: /root/.cache/pip/wheels/a6/90/95/f889ca4aa8b0e0c7f21c8470b6f5d6032f0390a3a141a9a3bd
Successfully built pickle5


In [2]:
import pickle5
import numpy as np
from sklearn.model_selection import train_test_split

# import the necessary packages
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers.experimental import preprocessing

import matplotlib.pyplot as plt

In [4]:
with open('data2.pickle', 'rb') as handle:
    data = pickle5.load(handle)

In [5]:
total_sample = len(data["joints_status"])
print(total_sample)

5040


In [6]:
X = np.array(data["ee_position"][elem])
y = np.array(data["joints_status"][elem])

num_elems = X.shape[0]
print(num_elems)
print(X[567])
print(y[567])

5040
[ 1.05499959 -0.69835187  1.01649764]
[  5.50657558 -12.332796   -25.13276388  -8.51898167  12.56663496
  18.85748715   7.85343962]


In [7]:
import plotly.graph_objects as go
import numpy as np

fig = go.Figure(data=[go.Scatter3d(x=X[:,0], y=X[:,1], z=X[:,2],
                                   mode='markers',
                                      marker=dict(
                                        size=2,
                                        colorscale='Viridis',   # choose a colorscale
                                        opacity=0.8
                                    ))])
fig.show()

# Remove all points aroud a eps *ball*

In [ ]:
index_to_delete = []
for i in range(num_elems):

  for j in range(num_elems):
    if (i != j) and np.linalg.norm(X[i]-X[j]) < 2e-2:
      if j not in index_to_delete:
        index_to_delete.append(j)
len(index_to_delete)

3302

In [ ]:
X = np.delete(X, index_to_delete, 0)
y = np.delete(y, index_to_delete, 0)
X.shape[0]

794

# Split dataset

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=True, random_state=128)
#X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.09, shuffle=True, random_state=128)

In [ ]:
X_train.shape

(96, 3)

In [ ]:
X[0]

array([0.27728424, 0.09114009, 1.43583368])

In [ ]:
y[0]

array([-1.5707, -1.5707, -1.5707, -1.5707, -1.5707, -1.5707,  0.    ])

# NN inverse K 

In [10]:
def create_model():
    model = Sequential()
    model.add(Dense(64, activation="relu"))
    model.add(Dense(7))
    sgd = tf.optimizers.SGD(0.001, momentum = 0.3, nesterov = True)
    adam = tf.optimizers.Adam(learning_rate=0.01)
    model.compile(loss="mean_squared_error", optimizer=adam)
    return model

In [16]:
# train the model using SGD
print("[INFO] training network...")
model = create_model()
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=2000, verbose = 2)

[INFO] training network...
Epoch 1/2000
142/142 - 1s - loss: 30.8929 - val_loss: 4.4770
Epoch 2/2000
142/142 - 0s - loss: 3.0854 - val_loss: 1.5592
Epoch 3/2000
142/142 - 0s - loss: 0.7824 - val_loss: 0.2571
Epoch 4/2000
142/142 - 0s - loss: 0.1525 - val_loss: 0.0998
Epoch 5/2000
142/142 - 0s - loss: 0.0942 - val_loss: 0.0876
Epoch 6/2000
142/142 - 0s - loss: 0.0888 - val_loss: 0.0845
Epoch 7/2000
142/142 - 0s - loss: 0.0871 - val_loss: 0.0839
Epoch 8/2000
142/142 - 0s - loss: 0.0863 - val_loss: 0.0829
Epoch 9/2000
142/142 - 0s - loss: 0.0856 - val_loss: 0.0839
Epoch 10/2000
142/142 - 0s - loss: 0.0850 - val_loss: 0.0837
Epoch 11/2000
142/142 - 0s - loss: 0.0847 - val_loss: 0.0821
Epoch 12/2000
142/142 - 0s - loss: 0.0843 - val_loss: 0.0818
Epoch 13/2000
142/142 - 0s - loss: 0.0842 - val_loss: 0.0843
Epoch 14/2000
142/142 - 0s - loss: 0.0845 - val_loss: 0.0817
Epoch 15/2000
142/142 - 0s - loss: 0.0846 - val_loss: 0.0830
Epoch 16/2000
142/142 - 0s - loss: 0.0847 - val_loss: 0.0822
Epoch

In [17]:
model.evaluate(X_test, y_test)

16/16 [==============================] - 0s 1ms/step - loss: 0.0455


0.04548944532871246

In [18]:
# Save the weights
model.save_weights('./checkpoints/nn_checkpoint_inverse_k')

In [19]:
!zip -r filename.zip ./checkpoints

  adding: checkpoints/ (stored 0%)
  adding: checkpoints/checkpoint (deflated 50%)
  adding: checkpoints/nn_checkpoint_inverse_k.data-00000-of-00001 (deflated 49%)
  adding: checkpoints/nn_checkpoint_inverse_k.index (deflated 58%)


In [20]:
print(model.predict(X_test[0].reshape(1,3)))
print(y_test[0])

[[  6.79978   -12.127266  -25.153694   -7.1806784  12.561437   19.745728
    7.861697 ]]
[  6.83984878 -12.32889057 -25.13264519  -7.7809233   12.56661208
  18.8630297    7.8534802 ]
